In [1]:
# import sys
# print(sys.executable)


In [2]:
# import platform
# print(platform.architecture())


In [3]:
# import jpype
# jpype.startJVM()  # Startet die JVM
# print("JVM erfolgreich gestartet!")
# jpype.shutdownJVM()


In [4]:
# from jpmml_evaluator import make_evaluator

# # Teste, ob die JVM erfolgreich gestartet werden kann
# try:
#     evaluator = make_evaluator('../models/analyzed_model.pmml')  # Verwende einen einfachen PMML-Pfad
#     evaluator.verify()
#     print("JVM erfolgreich gestartet und PMML-Modell erfolgreich geladen.")
# except Exception as e:
#     print(f"Fehler: {e}")


In [5]:
# print("START")
# import os
# import pandas as pd
# from pypmml import Model

# # Pfad zur PMML-Datei und zur CSV-Datei
# pmml_file_path = '/mnt/data/analyzed_model.pmml'
# csv_file_path = 'models/generated_data.csv'  # Passe den Pfad zur CSV-Datei bei Bedarf an

# # Funktion zum Laden des Modells mit verbesserter Fehlerbehandlung
# def load_model(file_path):
#     try:
#         # Modell mit pypmml laden
#         loaded_model = Model.load(file_path)
#         if loaded_model is not None:
#             print("Modell erfolgreich geladen.")
#             return loaded_model
#     except Exception as e:
#         print(f"Fehler beim Laden des PMML-Modells: {e}")
#     return None

# # Modell laden
# loaded_model = load_model(pmml_file_path)

# if loaded_model is not None:
#     # Lade die generierten Daten
#     data = pd.read_csv(csv_file_path)
    
#     # Vorverarbeitung: Sicherstellen, dass kategorische Variablen als solche erkannt werden
#     data['Geschlecht'] = data['Geschlecht'].astype('category')
#     data['Kategorie'] = data['Kategorie'].astype('category')

#     # Vorhersagen durchführen
#     predictions = loaded_model.predict(data)
#     print(predictions)  # Ergebnisse anzeigen
    
# else:
#     print("Das Modell konnte nicht geladen werden.")


In [6]:
# import pandas as pd
# from sklearn.tree import DecisionTreeRegressor  # Verwende DecisionTreeRegressor
# from sklearn2pmml.pipeline import PMMLPipeline
# from sklearn2pmml import sklearn2pmml
# from jpmml_evaluator import make_evaluator

# # PMML-Dateipfad und CSV-Dateipfad
# pmml_file_path = '../models/analyzed_model.pmml'
# csv_file_path = '../models/generated_data.csv'

# # Schritt 1: Modell erstellen, trainieren und als PMML speichern
# def train_and_save_model():
#     data = pd.read_csv(csv_file_path)
#     X = data.drop(columns=['Zielwert'])  # Ersetze 'Zielwert' mit dem tatsächlichen Zielnamen
#     y = data['Zielwert']  # Ersetze 'Zielwert' mit dem tatsächlichen Zielnamen

#     # Umwandeln der kategorischen Variablen in Dummy-Variablen
#     X = pd.get_dummies(X)

#     # Pipeline erstellen und trainieren
#     pipeline = PMMLPipeline([
#         ("regressor", DecisionTreeRegressor())  # Verwende DecisionTreeRegressor
#     ])
#     pipeline.fit(X, y)

#     # Modell in PMML-Datei speichern
#     sklearn2pmml(pipeline, pmml_file_path, with_repr=True)
#     print(f"Modell erfolgreich gespeichert als {pmml_file_path}.")

# # Schritt 2: Modell laden
# def load_model(file_path):
#     try:
#         evaluator = make_evaluator(file_path)
#         evaluator.verify()  # Überprüfen des Modells
#         print("Modell erfolgreich geladen mit jpmml-evaluator.")
#         return evaluator
#     except Exception as e:
#         print(f"Fehler beim Laden des PMML-Modells mit jpmml-evaluator: {e}")
#     return None

# # Schritt 3: Modell verwenden, um Vorhersagen zu machen
# def make_predictions(evaluator):
#     data = pd.read_csv(csv_file_path)
#     # Umwandeln der kategorischen Variablen in Dummy-Variablen
#     data = pd.get_dummies(data)

#     # Vorhersagen durchführen
#     predictions = evaluator.evaluate(data.to_dict(orient='records'))  # Daten ins Dictionary umwandeln

#     # Die Ergebnisse hier direkt verwenden
#     print("Vorhersagen:", predictions)  # Vorhersagen ausgeben

# # Hauptlogik
# if __name__ == "__main__":
#     train_and_save_model()  # Trainiere und speichere das Modell
#     loaded_model = load_model(pmml_file_path)  # Lade das Modell

#     if loaded_model is not None:
#         make_predictions(loaded_model)  # Mache Vorhersagen
#     else:
#         print("Das Modell konnte nicht geladen werden.")


In [7]:
import os
import numpy as np
import pandas as pd
from jpmml_evaluator import make_evaluator
from xgrove import grove  # Importiere die Grove-Klasse aus dem xgrove Paket

# Setze den Pfad für Graphviz
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'

def load_and_evaluate_model(pmml_file_path, csv_file_path):
    """Lädt das PMML-Modell und erstellt ein Surrogat-Modell mit xgrove."""
    
    # Lade die Trainingsdaten
    data = pd.read_csv(csv_file_path)
    # Definiere die Zielvariable (Name sollte zu deinem Datensatz passen)
    # target_variable = 'Zielwert'
    
    # Lade das PMML-Modell
    try:
        evaluator = make_evaluator(pmml_file_path)
        evaluator.verify()
        print("PMML-Modell erfolgreich geladen und verifiziert.")

        # Generiere Surrogat-Zielwerte für die Daten
        surrogate_targets = []
        for _, row in data.iterrows():
            input_data = row.to_dict()  # Erstelle ein Dictionary für eine Zeile
            prediction = evaluator.evaluate(input_data)  # Einzeldatensatz bewerten
            # print(prediction)
            surrogate_targets.append(prediction.get('Predicted_y', None))  # Nutze den Rückgabeschlüssel des Modells
            
        # Prüfe auf erfolgreiche Vorhersage
        if not surrogate_targets or None in surrogate_targets:
            raise ValueError("Fehler beim Erstellen der Surrogat-Zielwerte.")
        
        # Surrogat-Targets in den DataFrame aufnehmen
        # data['surrogatetarget'] = surrogate_targets

        # Instanziiere die Grove-Klasse mit Surrogat-Targets und dem Modell
        grove_model = grove(model=evaluator, data=data)
        
        # Berechnung des Surrogat-Modells durchführen
        grove_model.calculateGrove()

        print("Berechnungen abgeschlossen.")
        print(grove_model.get_result())

        # Zusätzliche Modellattribute anzeigen (falls vorhanden)
        model_classes = evaluator.getModelClasses()  # Modellklassen, falls definiert
        print(f"Model Klassen: {model_classes}")

    except Exception as e:
        print(f"Fehler beim Laden oder Verarbeiten des PMML-Modells: {e}")

if __name__ == "__main__":
    # Definiere die Datei- und CSV-Pfade
    pmml_file_path = '../models/linear_model.pmml'
    csv_file_path = '../models/generated_data.csv'

    load_and_evaluate_model(pmml_file_path, csv_file_path)


its upgraded
feature_index:  6
feature_index:  7
feature_index:  6
feature_index:  1
feature_index:  1
feature_index:  7
feature_index:  0
feature_index:  6
feature_index:  3
feature_index:  7
feature_index:  0
feature_index:  7
feature_index:  8
feature_index:  4
feature_index:  1
feature_index:  9
feature_index:  7
feature_index:  4
feature_index:  3
feature_index:  4
feature_index:  0
feature_index:  0
feature_index:  4
feature_index:  1
feature_index:  2
feature_index:  6
feature_index:  3
feature_index:  5
feature_index:  6
feature_index:  7
feature_index:  6
feature_index:  1
feature_index:  1
feature_index:  7
feature_index:  0
feature_index:  6
feature_index:  3
feature_index:  7
feature_index:  0
feature_index:  7
feature_index:  8
feature_index:  4
feature_index:  1
feature_index:  9
feature_index:  7
feature_index:  4
feature_index:  3
feature_index:  4
feature_index:  0
feature_index:  0
feature_index:  4
feature_index:  1
feature_index:  2
feature_index:  6
feature_index: 